In [ ]:
import os
import torch
import pandas as pd
import numpy as np
import cv2
import glob
import random
from tqdm.notebook import tqdm

In [ ]:
cd /content/drive/MyDrive/2021Hanium_deepfakeDetectionApplication/Kim_jeeyoung_folder/deepfake-detection-challenge/train_images

/content/drive/MyDrive/2021Hanium_deepfakeDetectionApplication/Kim_jeeyoung_folder/deepfake-detection-challenge/train_images


In [ ]:
cd /content/

/content


In [ ]:
!git clone https://github.com/HongguLiu/Deepfake-Detection

Cloning into 'Deepfake-Detection'...
remote: Enumerating objects: 108, done.
remote: Counting objects: 100% (67/67), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 108 (delta 34), reused 6 (delta 2), pack-reused 41
Receiving objects: 100% (108/108), 129.93 MiB | 21.71 MiB/s, done.
Resolving deltas: 100% (44/44), done.


In [ ]:
!pip install pretrainedmodels==0.7.4

     |████████████████████████████████| 61kB 3.3MB/s 
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-cp37-none-any.whl size=60966 sha256=a6975b9e7d8d36788fc6537f2ad8bab97ed8c86fabf0a8c1b8971a5ad3ea8fd0
  Stored in directory: /root/.cache/pip/wheels/69/df/63/62583c096289713f22db605aa2334de5b591d59861a02c2ecd
Successfully built pretrainedmodels


In [ ]:
import sys
sys.path.insert(0, "/content/Deepfake-Detection/network/")

In [ ]:
sys.path.insert(1,'/content/Deepfake-Detection/')

In [ ]:
!mkdir -p /home/ondyari/.torch/models

In [ ]:
import gdown
url = 'https://drive.google.com/uc?id=1PDGYCwVv-ANPHEFzbDKo9rPmhAFRxrrM'

output = '/home/ondyari/.torch/models/xception-b5690688.pth'
gdown.download(url,output,quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1PDGYCwVv-ANPHEFzbDKo9rPmhAFRxrrM
To: /home/ondyari/.torch/models/xception-b5690688.pth
91.7MB [00:00, 322MB/s]


'/home/ondyari/.torch/models/xception-b5690688.pth'

In [ ]:
def videoReader(path,n_frames=10,SAVE_DIR=None,detector = None):
    v_cap = cv2.VideoCapture(path)
    v_len = int(v_cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fourcc = cv2.VideoWriter_fourcc(*'MJPG')

        # Pick 'n_frames' evenly spaced frames to sample
    if n_frames is None:
        sample = np.arange(0, v_len)
    else:
        sample = np.linspace(0, v_len - 1, n_frames).astype(int)
    face_detector = detector()
    filename = path.split('/')[-1]
    # Loop through frames
    faces = []
    frames = []
    for j in range(v_len):
        success = v_cap.grab()
        if j in sample:
            # Load frame
            #cv2.VideoCapture.retrieve() : grab한 frame을 decode해서 반환
            success, frame = v_cap.retrieve()
            if not success:
                continue
            #frame = cv2.resize(frame,(240,240))
            frames.append(frame)
            face = face_detector.detect_faces(frame)
            if len(face) == 0 : continue
            print(face)
            size = int(face[0]['box'][2] * face[0]['box'][3])
            faces.append((face,size,frame))

    if len(faces) != 0 :
      sorted(faces,key=lambda face: face[1])

      img = faces[0]
      x, y, width, height= img[0][0]['box']
      cropped_face = (img[2])[y:y+height, x:x+width]
      filename = filename.split('.')[0]
      cv2.imwrite(SAVE_DIR + filename +'.jpg',cropped_face)

In [ ]:
TRAIN_DIR = '/content/drive/MyDrive/2021Hanium_deepfakeDetectionApplication/train_sample_videos/'
SAVE_DIR = '/content/drive/MyDrive/2021Hanium_deepfakeDetectionApplication/Kim_jeeyoung_folder/deepfake-detection-challenge/train_images/'
folders = os.listdir(TRAIN_DIR)
folders.remove("metadata.json")

In [ ]:
!pip install mtcnn
from mtcnn import MTCNN

In [ ]:
all_train_videos = glob.glob(os.path.join(TRAIN_DIR, '*.mp4'))
for path in tqdm(all_train_videos):
    videoReader(path,10,SAVE_DIR,MTCNN)

KeyboardInterrupt: ignored

In [ ]:
all_train_images = glob.glob(os.path.join(SAVE_DIR, '*.jpg'))

import json

metadata_path = TRAIN_DIR + 'metadata.json'
with open(metadata_path, 'r') as f :
  metadata = json.load(f)

In [ ]:
from PIL import Image
from torch.utils.data import Dataset
import os
import random


class myDataset(Dataset):
    def __init__(self, train_list, metadata,transform=None, target_transform=None):
        #fh = open(txt_path, 'r')
        imgs = []
        paths = []
        for path in tqdm(train_list):
          file_name = path.split('/')[-1]
          file_name = file_name.split('.')[0] + '.mp4'

          if metadata[file_name]['label'] == 'FAKE': label = 1
          else :label = 0

          imgs.append((path, int(label)))

        self.imgs = imgs
        self.transform = transform
        self.target_transform = target_transform

    def __getitem__(self, index):
        fn, label = self.imgs[index]
        img = Image.open(fn).convert('RGB')

        if self.transform is not None:
            img = self.transform(img)

        return img, label

    def getPath(self,index):
      return (self.imgs[index])[0]

    def __len__(self):
        return len(self.imgs)

In [ ]:
from dataset.transform import xception_default_data_transforms
train_dataset = myDataset(train_list = all_train_images, metadata = metadata,transform = xception_default_data_transforms['train'])

In [ ]:
X = []
y = []
for i in tqdm(range(len(train_dataset))) :
  path, label = train_dataset.getPath(i), (train_dataset[i])[1]
  X.append(path)
  y.append(label)

In [ ]:
def shuffle(X,y):
    new_train=[]
    for m,n in zip(X,y):
        new_train.append([m,n])
    random.shuffle(new_train)
    X,y=[],[]
    for x in new_train:
        X.append(x[0])
        y.append(x[1])
    return X,y

In [ ]:
x,Y = shuffle(X,y)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val =train_test_split(x,Y,test_size=0.15,random_state=128)

In [ ]:
print(len(X_train),len(X_val))

340 60


In [ ]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
!nvidia-smi

Thu Jul 15 15:00:57 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import pandas
np_X, np_Y = np.array(X_train), np.array(y_train)
d = np.array([np_X,np_Y])
c = pandas.DataFrame(np.transpose(d),columns = ['filename','label'])

                                              filename label
0    /content/drive/MyDrive/2021Hanium_deepfakeDete...     1
1    /content/drive/MyDrive/2021Hanium_deepfakeDete...     0
2    /content/drive/MyDrive/2021Hanium_deepfakeDete...     0
3    /content/drive/MyDrive/2021Hanium_deepfakeDete...     1
4    /content/drive/MyDrive/2021Hanium_deepfakeDete...     1
..                                                 ...   ...
335  /content/drive/MyDrive/2021Hanium_deepfakeDete...     1
336  /content/drive/MyDrive/2021Hanium_deepfakeDete...     1
337  /content/drive/MyDrive/2021Hanium_deepfakeDete...     0
338  /content/drive/MyDrive/2021Hanium_deepfakeDete...     0
339  /content/drive/MyDrive/2021Hanium_deepfakeDete...     0

[340 rows x 2 columns]


In [ ]:
label_count = c.groupby('label').count()['filename']

global pos_weight
pos_weight = torch.ones([1]) * label_count[0]/label_count[1]
print('pos_weight:', pos_weight)

pos_weight: tensor([0.2274])


In [ ]:
model_path2 = '/content/drive/MyDrive/2021Hanium_deepfakeDetectionApplication/faceforensics++_models/faceforensics++_models_subset/full/xception/full_raw.p'

In [ ]:
import torch
import torch.nn as nn
import torchvision
from torch.utils.data import DataLoader
import torch.optim as optim
from torch.optim import lr_scheduler
import argparse
import os
import cv2

from network.models import model_selection
from mesonet import Meso4, MesoInception4
from dataset.transform import xception_default_data_transforms
from dataset.mydataset import MyDataset

def train(output_name='xception_weights', output_order = 1,train=[],val=[],meta = None, epoches = 20,batch_size = 38,  model_path=None):
  folder_name = 'train' + '_' + str(output_order)
  output_path = os.path.join(model_path,folder_name)
  os.mkdir(output_path)

  torch.backends.cudnn.benchmark = True
  train_dataset = myDataset(train_list = train, metadata = meta,transform = xception_default_data_transforms['train'])
  val_dataset = myDataset(train_list = val, metadata = meta,transform=xception_default_data_transforms['val'])
  train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=False, num_workers=8)
  val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True, drop_last=False, num_workers=8)
  train_dataset_size = len(train_dataset)
  val_dataset_size = len(val_dataset)

  model = model_selection(modelname='xception', num_out_classes=2, dropout=0.5)
  #model = torch.load(model_path2)
  model = model.cuda()
  criterion = nn.CrossEntropyLoss()
  optimizer = optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08)
  scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

  model = nn.DataParallel(model)
  best_model_wts = model.state_dict()
  best_acc = 0.0
  iteration = 0

  for epoch in tqdm(range(epoches)):
    print('Epoch {}/{}'.format(epoch+1, epoches))
    #print(best_model_wts)
    print('-'*10)
    model.train()
    train_loss = 0.0
    train_corrects = 0.0
    val_loss = 0.0
    val_corrects = 0.0
    for (image,labels) in train_loader:
      iter_loss = 0.0
      iter_corrects = 0.0
      image = image.cuda()
      labels = labels.cuda()
      optimizer.zero_grad()
      outputs = model(image)
      _, preds = torch.max(outputs.data, 1)
      loss = criterion(outputs, labels)
      loss.backward()
      optimizer.step()
      iter_loss = loss.data.item()
      train_loss += iter_loss
      iter_corrects = torch.sum(preds == labels.data).to(torch.float32)
      train_corrects += iter_corrects
      iteration += 1
      if not (iteration % 20):
        print('iteration {} train loss: {:.4f} Acc: {:.4f}'.format(iteration, iter_loss / batch_size, iter_corrects / batch_size))
    epoch_loss = train_loss / train_dataset_size
    epoch_acc = train_corrects / train_dataset_size
    print('epoch train loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc))

    model.eval()
    with torch.no_grad():
      for (image, labels) in val_loader:
        image = image.cuda()
        labels = labels.cuda()
        outputs = model(image)
        _, preds = torch.max(outputs.data, 1)
        loss = criterion(outputs, labels)
        val_loss += loss.data.item()
        val_corrects += torch.sum(preds == labels.data).to(torch.float32)
      epoch_loss = val_loss / val_dataset_size
      epoch_acc = val_corrects / val_dataset_size
      print('epoch val loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc))
      if epoch_acc > best_acc:
        best_acc = epoch_acc
        best_model_wts = model.state_dict()
      scheduler.step()
		#if not (epoch % 40):
    torch.save(model.state_dict(), output_path + '/' + output_name + str(epoch) + ".pkl")
  print('Best val Acc: {:.4f}'.format(best_acc))
  model.load_state_dict(best_model_wts)
  torch.save(model.state_dict(), output_path + '/' + output_name + '_' + "best.pkl")

In [ ]:
SAVE_DIR = '/content/drive/MyDrive/2021Hanium_deepfakeDetectionApplication/Kim_jeeyoung_folder/model/firstXceptionRunModel/'

In [ ]:
train(output_name='xception_weights', output_order = 3,train = X_train, val= X_val,meta = metadata, model_path=SAVE_DIR)


Using dropout 0.5


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


RuntimeError: ignored